# 비속어 처리

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate # load_metric이 datasets >= 2.17 부터 제거됨

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
# from datasets import load_metric

from evaluate import load


# Load any dataset of choice for training.
#hate_dataset = load_dataset("SetFit/toxic_conversations")        # --> 실제 훈련 시 사용
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)

#tokenized_train = hate_dataset["train"].map(preprocess_function, batched=True)     # --> 실제 훈련 시 사용
#tokenized_test = hate_dataset["test"].map(preprocess_function, batched=True)       # --> 실제 훈련 시 사용

tmp_train = load_dataset("SetFit/toxic_conversations", split="train[0:10]")   # --> 실제 훈련시 삭제
tmp_test = load_dataset("SetFit/toxic_conversations", split="test[0:10]")   # --> 실제 훈련시 삭제
tokenized_train = tmp_train.map(preprocess_function, batched=True)
tokenized_test = tmp_test.map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

In [ ]:
tokenized_train

In [ ]:
def compute_metrics(eval_pred):
  load_accuracy = load("accuracy")
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
  return {"accuracy": accuracy}

training_args = TrainingArguments(
    output_dir="hate",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # num_train_epochs=2,
    num_train_epochs=0.2,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
from google.colab import userdata

# OpenAIKey = userdata.get("OPENAI_KEY") # OPENAI KEY
client = OpenAI(api_key="sk-proj-")

def detect_hate(input_text):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=0,
    messages = [
          {"role": "system",
           "content" :
           """You are an expert content moderator."""},
          {"role": "user",
           "content" : "Is this hate speech?  ```%s```" %(input_text)}])
  res = response.choices[0].message.content
  return res


In [ ]:
detect_hate("It is about time for all illegals to go back to their country of origin and keep our freeway open and prevent heavy traffic.")